In [1]:
import os
import sys
import copy
pdir = os.path.dirname(os.getcwd())
sys.path.append(pdir)

import torch
import numpy as np

import utils
import wandb

model_dir = 'models'
data_dir = 'tree_points.pt'

In [2]:
trees_data = torch.load(data_dir)

In [3]:
sweep_config = {
  "name":"laser-trees-bayes",
  "method":"bayes",
  "metric":{
      "name":"Best_acc",
      "goal":"maximize"
  },
  "parameters":{
    "loss_fn":{
      "values":["smooth-loss", "cross-entropy"]
    },
    "lr_init":{
      "min": 0.0001,
      "max": 0.001
    },
    "lr_step":{
      "min": 10,
      "max": 100
    },
    "lr_gamma":{
      "min": 0.1,
      "max": 0.9
    },
    "max_rotation":{
      "min":0,
      "max":2*np.pi
    },
    "max_translation":{
      "min":0,
      "max":2.0
    },
    "jitter_std":{
      "distribution":"log_uniform",
      "min":-7.0,
      "max":-2.0
    },
    "min_scale":{
      "min":0.3,
      "max":1
    },
    "max_scale":{
      "min":1,
      "max":2
    }
  }
}

In [4]:
def interface_to_train():
    with wandb.init() as run:
        params={
        "dataset_type":type(trees_data),
        "batch_size":128,
        "validation_split":.15,
        "test_split":.15,
        "shuffle_dataset":True,
        "random_seed":0,
        "learning_rate":[wandb.config["lr_init"], wandb.config["lr_step"], wandb.config["lr_gamma"]],  #[init, step_size, gamma] for scheduler
        "momentum":0.9, #Only used for sgd
        "epochs":150,
        "loss_fn":wandb.config["loss_fn"],
        "optimizer":"adam",
        "voting":"None",
        "train_sampler":"balanced",

        "model":"SimpleView",

        "image_dim":256,
        "camera_fov_deg":90,
        "f":1,
        "camera_dist":1.4,
        "depth_averaging":"min",
        "soft_min_k":50,
        "num_views":6,

        "transforms":['rotation','translation','scaling'], #,'translation'
        "min_rotation":0,
        "max_rotation":wandb.config["max_rotation"],
        "min_translation":0,
        "max_translation":wandb.config["max_translation"],
        "jitter_std":wandb.config["jitter_std"],
        "min_scale":wandb.config["min_scale"],
        "max_scale":wandb.config["max_scale"],

        "species":["QUEFAG", "PINNIG", "QUEILE", "PINSYL", "PINPIN"],
        "data_resolution":"2.5cm"
        }
        
        utils.train(data_dir=data_dir, model_dir=model_dir, params=params, init_wandb=False)
    return

In [6]:
sweep_id = '3gczm5qj'
#sweep_id = wandb.sweep(sweep_config, project='laser-trees-bayes')

count = 3
wandb.agent(sweep_id, project='laser-trees-bayes', function=interface_to_train, count=count)

wandb: Agent Starting Run: ymf3f79x with config:
wandb: 	jitter_std: 0.06921816699542221
wandb: 	loss_fn: smooth-loss
wandb: 	lr_gamma: 0.35164817059800135
wandb: 	lr_init: 0.0008212813435818885
wandb: 	lr_step: 11
wandb: 	max_rotation: 3.9051038810599707
wandb: 	max_scale: 2
wandb: 	max_translation: 2
wandb: 	min_scale: 0.9312167148500241
wandb: Currently logged in as: mja2106 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Config item 'loss_fn' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_rotation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_translation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'jitter_std' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_scale' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_scale' was locked by 'sweep' (ignored update).


QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485
Removing: DEAD
Removing: QUERCUS
Removing: NA
Removing: JUNIPE
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Shuffling dataset...
Using balanced sampling...
Using smooth-loss
Optimizing with AdaM...
Using step LR scheduler...


/home/matt/anaconda3/envs/laser-trees/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,     5] loss: 4.022
[1,    10] loss: 3.877
[1,    15] loss: 3.854
[1,    20] loss: 3.900
OVERALL: Got 52 / 372 with accuracy 13.98
PINNIG: Got 26/93 with accuracy 27.96
PINPIN: Got 12/18 with accuracy 66.67
PINSYL: Got 14/40 with accuracy 35.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[2,     5] loss: 3.859
[2,    10] loss: 3.839
[2,    15] loss: 3.839
[2,    20] loss: 3.795
OVERALL: Got 18 / 372 with accuracy 4.84
PINNIG: Got 1/93 with accuracy 1.08
PINPIN: Got 17/18 with accuracy 94.44
PINSYL: Got 0/40 with accuracy 0.00
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[3,     5] loss: 3.795
[3,    10] loss: 3.805
[3,    15] loss: 3.740
[3,    20] loss: 3.708
OVERALL: Got 29 / 372 with accuracy 7.80
PINNIG: Got 0/93 with accuracy 0.00
PINPIN: Got 14/18 with accuracy 77.78
PINSYL: Got 15/40 with accuracy 37.50
QUEFAG: Got 0/177 with accuracy 0.00
QUEILE: Got 0/44 with accuracy 0.00
[4,     5] loss: 3.712
[4,    10] loss: 3.705

...

wandb: Agent Starting Run: t1a4r102 with config:
wandb: 	jitter_std: 0.0009529360613864005
wandb: 	loss_fn: cross-entropy
wandb: 	lr_gamma: 0.5515068937465009
wandb: 	lr_init: 0.00023600223283301943
wandb: 	lr_step: 88
wandb: 	max_rotation: 2.3686366791744415
wandb: 	max_scale: 2
wandb: 	max_translation: 1
wandb: 	min_scale: 0.4534051994331355
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Config item 'loss_fn' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_rotation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_translation' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'jitter_std' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'min_scale' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_scale' was locked by 'sweep' (ignored update).


QUEFAG     1116
PINNIG      581
QUEILE      364
PINSYL      277
PINPIN      140
NA            2
JUNIPE        2
QUERCUS       2
DEAD          1
Name: sp, dtype: int64
Species:  ['DEAD', 'JUNIPE', 'NA', 'PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE', 'QUERCUS']
Labels:  tensor([8, 3, 6,  ..., 7, 3, 6])
Total count:  2485
Removing: DEAD
Removing: QUERCUS
Removing: NA
Removing: JUNIPE
Train Dataset:
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Validation Dataset (should match):
QUEFAG    1116
PINNIG     581
QUEILE     364
PINSYL     277
PINPIN     140
Name: sp, dtype: int64
Species:  ['PINNIG', 'PINPIN', 'PINSYL', 'QUEFAG', 'QUEILE']
Labels:  tensor([0, 3, 3,  ..., 4, 0, 3])
Total count:  2478

Shuffling dataset...
Using balanced sampling...
Using cross-entropy loss...
Optimizing with AdaM...
Using step LR schedul

PINNIG Accuracy,0.68817
PINPIN Accuracy,0.66667
PINSYL Accuracy,0.475
QUEFAG Accuracy,0.76271
QUEILE Accuracy,0.77273
Best_acc,0.76075
Best_min_acc,0.5914
Train Loss,3.88325
Validation Loss,0.73352
Train Accuracy,0.77078
Validation Accuracy,0.70968


PINNIG Accuracy,▁▅▂▅▅▂▇▄▃▄▂▂▄▃▅▆▃▅▅▃▃▅▅▃▅▆▆▄▅█▅█▂▄▅▄▆▄▅▇
PINPIN Accuracy,█▆▃▁▂▅▂▃▇▃█▂▅▅▂▅▁▅▃▅█▄▅██▃▇▆▃▃█▂▆▃▅█▆▆▆▆
PINSYL Accuracy,▁▁▂▆▄▅▃▃▅▅▅▂▂▆▇▄▅▅▄▆▄▃▇▅▅▆▄█▇▃▅▅▅▃▇▄▆▇▅▅
QUEFAG Accuracy,▁▃▇▂▃█▂▆▆▅▆▇█▇▁▆█▃▆▆▆▇▇▇▇▇▇▆█▇█▆█▆▆▇▅▇▇▇
QUEILE Accuracy,▁▅▆▅█▂▄▇▅█▆▇▄▇▃▆▅▅▇▇▇▅▆▇▅▇▆▅▅▆▅▇▅▇▆▇▇▆▇▇
Best_acc,▁▄▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
Best_min_acc,▁▂▂▂▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇████
Train Loss,█▇▆▇▆▆▆▄▄▃▅▆▅▃█▃▄▅▃▃▃▃▂▃▂▁▂▄▂▂▁▂▃▅▁▂▁▁▁▁
Validation Loss,▅▄▂▅▅▃▅▃▂▃▃▄▄▂█▂▂▅▂▂▃▃▁▂▂▁▁▃▁▂▁▂▃▄▂▂▂▂▂▁
Train Accuracy,▁▂▄▃▄▄▄▅▅▆▅▅▅▆▃▇▆▅▇▆▆▆▇▇▇██▆▇▇█▇▇▆█▇████
Validation Accuracy,▁▄▆▄▅▆▄▇▆▆▆▆▇▇▃▇▇▅▇▇▇▇█▇▇██▆████▇▇▇▇▇▇▇█
